In [1]:
import requests
import json
import os
from bs4 import BeautifulSoup
from datetime import datetime
import newspaper

In [53]:
source = 'empirenews'
month = '03'
days = [str(i) for i in list(range(7,10))]

dates = []
for day in days:
    dates.append("{0}-0{1}".format(month, day))

In [54]:
for date in dates:
    # Load HTML
    filepath = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/{1}.html'.format(source,date)
    with open(filepath) as f:
        soup = BeautifulSoup(f, 'lxml')

    # Pull out article links
    links = []
    i=0

    for p in soup.findAll('p', attrs={"class":"date"}):
        pubdate = str(datetime.strptime(p.text, "%B %d, %Y"))
        if date in pubdate:
            for header in soup.findAll('h2'):
                for link in header.findAll('a'):
                    links.append(link.get('href'))

    links = set(links)
    links = list(links)
    
    for link in links:
        link = newspaper.Article(link)
        link.download()
        html = link.html
        link.parse()
        date_longform = '{0}-2017'.format(date)
        title = link.title.replace('Infostormer.Com   |  ', '')
        
        article = {}
        article["html"] = html
        article["title"] = title
        article["url"] = link.url
        article["date"] = date_longform
        article["source"] = source
        article["text"] = link.text
        article["images"] = list(link.images)
        article["videos"] = link.movies

        outfile = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/articles/{2}/article{1}_{2}.txt'.format(source,i,date)
        os.makedirs(os.path.dirname(outfile), exist_ok=True)
        i+=1

        with open(outfile, 'w') as f:
            json.dump(article, f)

In [ ]:
# Test that articles saved correctly
with open('/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/infostormer/articles/03-07/article0_03-07.txt') as json_data:
    d = json.load(json_data)
    
print(d["title"])
print()
print(d["source"])
print()
print(d["url"])
print()
print(d["date"])
print()
print(d["text"])
print()
print(d["images"])
print()
print(d["videos"])
print()
print(d["html"])